In [7]:
FROM python:3.12-slim

RUN apt-get update && \
    apt-get install -y wget libgomp1

WORKDIR /app

RUN wget https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.16.0+-x64-linux.tar.gz && \
    tar -xzvf ncbi-blast-2.16.0+-x64-linux.tar.gz && \
    mv ncbi-blast-2.16.0+ /usr/local/ && \
    rm ncbi-blast-2.16.0+-x64-linux.tar.gz

ENV PATH="/usr/local/ncbi-blast-2.16.0+/bin:$PATH"

RUN wget https://ftp.ncbi.nlm.nih.gov/blast/db/v5/swissprot.tar.gz && \
    mkdir -p /db/swissprot && mv swissprot* /db/swissprot && \
    tar -xzvf /db/swissprot/swissprot.tar.gz -C /db/swissprot

COPY . /app

SyntaxError: invalid syntax (<ipython-input-7-16cedae63e7c>, line 1)

In [2]:
import os
import subprocess
import matplotlib.pyplot as plt
!pip install biopython
from Bio import SeqIO
from Bio.Blast import NCBIXML

def read_fasta(filepath):
    try:
        return [(record.id, str(record.seq)) for record in SeqIO.parse(filepath, "fasta")]
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
        return []

def find_orfs_in_sequence(sequence_data, sequence_identifier):
    start_codon = "ATG"
    stop_codons = {"TAA", "TAG", "TGA"}
    found_orfs = []
    current_index = 0
    while current_index <= len(sequence_data) - 3:
        if sequence_data[current_index:current_index+3] == start_codon:
            orf_start_position = current_index
            current_index += 3
            while current_index <= len(sequence_data) - 3:
                codon = sequence_data[current_index:current_index+3]
                if codon in stop_codons:
                    orf_end_position = current_index + 3
                    reading_frame = (orf_start_position % 3) + 1
                    found_orfs.append({
                        "id": f"{sequence_identifier}_ORF{len(found_orfs)+1}",
                        "sequence": sequence_data[orf_start_position:orf_end_position],
                        "length": orf_end_position - orf_start_position,
                        "source": sequence_identifier,
                        "start": orf_start_position,
                        "end": orf_end_position,
                        "frame": reading_frame
                    })
                    break
                current_index += 3
        else:
            current_index += 1
    return found_orfs

def write_gff_file(orf_list, gff_filepath):
    with open(gff_filepath, "w") as f:
        f.write("##gff-version 3\n")
        f.write(f"##date: {os.stat(gff_filepath).st_mtime if os.path.exists(gff_filepath) else 'unknown'}\n")
        for orf in orf_list:
            f.write(
                f"{orf['source']}\tORF_Pipeline\tCDS\t{orf['start']+1}\t{orf['end']}\t.\t+\t.\t"
                f"ID={orf['id']};length={orf['length']};frame={orf['frame']}\n"
            )

def write_fasta_file(orf_list, fasta_filepath):
    with open(fasta_filepath, "w") as f:
        for orf in orf_list:
            f.write(f">{orf['id']}\n{orf['sequence']}\n")

def run_orf_analysis(input_fasta_file, output_gff_file, output_fasta_file):
    sequences_data = read_fasta(input_fasta_file)
    all_extracted_orfs = []

    for index, (seq_id, sequence_string) in enumerate(sequences_data):
        orfs_for_this_seq = find_orfs_in_sequence(sequence_string, f"Sequence_{index + 1}")
        all_extracted_orfs.extend(orfs_for_this_seq)

    write_gff_file(all_extracted_orfs, output_gff_file)
    write_fasta_file(all_extracted_orfs, output_fasta_file)
    print(f"Extracted {len(all_extracted_orfs)} ORFs from {len(sequences_data)} sequences.")

    return all_extracted_orfs

def plot_orf_lengths(orfs_data, plot_filepath="orf_lengths_histogram.png"):
    lengths = [orf['length'] for orf in orfs_data]
    if lengths:
        plt.hist(lengths, bins=50, edgecolor='black')
        plt.title("Distribution of ORF Lengths")
        plt.xlabel("ORF Length (bp)")
        plt.ylabel("Frequency")
        plt.savefig(plot_filepath)
        plt.close()
        print(f"ORF length histogram saved to {plot_filepath}")
    else:
        print("No ORFs found to plot.")

# BLAST+ Validation
def run_blast_comparison(query_fasta_path, db_path, output_blast_path):
    print(f"Running BLASTp with query: {query_fasta_path}, database: {db_path}")
    command = [
        "blastp",
        "-query", query_fasta_path,
        "-db", db_path,
        "-outfmt", "5",
        "-taxids", "9606",
        "-evalue", "1e-5",
        "-out", output_blast_path
    ]
    try:
        subprocess.run(command, check=True, capture_output=True, text=True)
        print(f"BLASTp completed successfully. Results saved to {output_blast_path}")
    except subprocess.CalledProcessError as e:
        print(f"BLASTp failed with error: {e}")
        print(f"STDOUT: {e.stdout}")
        print(f"STDERR: {e.stderr}")
        raise
    except FileNotFoundError:
        print("Error: 'blastp' command not found. Ensure BLAST+ is installed and in your system's PATH.")
        print("If using Docker, ensure your Dockerfile configures the PATH correctly and you are running within the container.")
        raise

def parse_blast_xml_results(blast_xml_path):
    print(f"Parsing BLAST XML results from {blast_xml_path}")
    blast_results = {}
    try:
        with open(blast_xml_path, "r") as result_handle:
            blast_records = NCBIXML.parse(result_handle)
            for blast_record in blast_records:
                query_id = blast_record.query.split(" ", 1)[0]
                best_hit = None
                best_evalue = float('inf')

                for alignment in blast_record.alignments:
                    for hsp in alignment.hsps:
                        if hsp.expect < best_evalue:
                            best_evalue = hsp.expect
                            best_hit = {
                                "subject_id": alignment.accession,
                                "evalue": hsp.expect,
                                "query_start_in_orf": hsp.query_start,
                                "query_end_in_orf": hsp.query_end,
                            }
                if best_hit:
                    blast_results[query_id] = best_hit
    except FileNotFoundError:
        print(f"Error: BLAST XML file '{blast_xml_path}' not found.")
    except Exception as e:
        print(f"Error parsing BLAST XML: {e}")
    return blast_results

def update_gff_with_cds(original_gff_path, updated_gff_path, orf_data, blast_validation_results):
    print(f"Updating GFF file {original_gff_path} with CDS information...")
    updated_lines = []
    with open(original_gff_path, 'r') as f_in:
        for line in f_in:
            updated_lines.append(line.strip())

    orf_info_map = {orf['id']: orf for orf in orf_data}

    for orf_id, blast_hit in blast_validation_results.items():
        if orf_id in orf_info_map:
            original_orf = orf_info_map[orf_id]

            cds_start_on_contig = original_orf['start'] + blast_hit['query_start_in_orf']
            cds_end_on_contig = original_orf['start'] + blast_hit['query_end_in_orf']

            if cds_start_on_contig > cds_end_on_contig:
                 cds_start_on_contig, cds_end_on_contig = cds_end_on_contig, cds_start_on_contig

            updated_lines.append(
                f"{original_orf['source']}\tBLAST_Validation\tCDS\t{cds_start_on_contig}\t{cds_end_on_contig}\t"
                f".\t+\t.\tParent={orf_id};BlastHitID={blast_hit['subject_id']}"
            )

            if blast_hit['query_start_in_orf'] > 1:
                utr5_start = original_orf['start'] + 1
                utr5_end = original_orf['start'] + blast_hit['query_start_in_orf'] - 1
                updated_lines.append(
                    f"{original_orf['source']}\tORF_Pipeline\t5_prime_UTR\t{utr5_start}\t{utr5_end}\t"
                    f".\t+\t.\tParent={orf_id}"
                )
            if blast_hit['query_end_in_orf'] < original_orf['length']:
                utr3_start = original_orf['start'] + blast_hit['query_end_in_orf'] + 1
                utr3_end = original_orf['end']
                updated_lines.append(
                    f"{original_orf['source']}\tORF_Pipeline\t3_prime_UTR\t{utr3_start}\t{utr3_end}\t"
                    f".\t+\t.\tParent={orf_id}"
                )

    with open(updated_gff_path, 'w') as f_out:
        for line in updated_lines:
            f_out.write(line + '\n')
    print(f"Updated GFF with CDS features saved to {updated_gff_path}")

def classify_orfs_by_blast(orf_data, blast_validation_results):
    validated_orfs = []
    unvalidated_orfs = []
    for orf in orf_data:
        if orf['id'] in blast_validation_results:
            validated_orfs.append(orf)
        else:
            unvalidated_orfs.append(orf)
    return validated_orfs, unvalidated_orfs

def plot_true_false_positive_lengths(validated_orfs, unvalidated_orfs, plot_filepath="orf_true_false_positive_lengths.png"):
    validated_lengths = [orf['length'] for orf in validated_orfs]
    unvalidated_lengths = [orf['length'] for orf in unvalidated_orfs]

    plt.figure(figsize=(10, 6))
    if validated_lengths:
        plt.hist(validated_lengths, bins=50, alpha=0.7, label='Validated ORFs (True Positives)', color='skyblue', edgecolor='black')
    if unvalidated_lengths:
        plt.hist(unvalidated_lengths, bins=50, alpha=0.7, label='Unvalidated ORFs (False Positives)', color='salmon', edgecolor='black')

    if not validated_lengths and not unvalidated_lengths:
        print("No ORFs to plot for true/false positives.")
        plt.close()
        return

    plt.title("Distribution of ORF Lengths: Validated vs. Unvalidated")
    plt.xlabel("ORF Length (bp)")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(axis='y', alpha=0.75)
    plt.savefig(plot_filepath)
    plt.close()
    print(f"True/False positive length histogram saved to {plot_filepath}")

def calculate_expected_orf_length_geometric():
    probability_of_stop_codon = 3 / 64
    expected_length_codons = 1 / probability_of_stop_codon
    expected_length_bp = expected_length_codons * 3
    print(f"Expected ORF length (codons, geometric model): {expected_length_codons:.2f}")
    print(f"Expected ORF length (base pairs, geometric model): {expected_length_bp:.2f}")
    return expected_length_codons, expected_length_bp

# Main Execution
if __name__ == "__main__":
    input_fasta = "Homo_sapiens_cdna_assembed.fasta"
    output_gff = "extracted_orfs.gff"
    output_fasta = "extracted_orfs.fasta"
    updated_gff_with_cds = "extracted_orfs_with_cds.gff"

    blast_db_path = "/db/swissprot/swissprot"
    blast_output_xml = "blast_results.xml"

    # Step 1: Extract ORFs and generate initial files
    final_orfs = run_orf_analysis(input_fasta, output_gff, output_fasta)

    # Step 2: Plot overall ORF lengths
    plot_orf_lengths(final_orfs)

    # Step 3: Validate ORFs with BLAST+
    try:
        run_blast_comparison(output_fasta, blast_db_path, blast_output_xml)
        blast_hits = parse_blast_xml_results(blast_output_xml)

        update_gff_with_cds(output_gff, updated_gff_with_cds, final_orfs, blast_hits)

        validated_orfs, unvalidated_orfs = classify_orfs_by_blast(final_orfs, blast_hits)
        plot_true_false_positive_lengths(validated_orfs, unvalidated_orfs)

        print(f"\nSummary of BLAST validation:")
        print(f"Total predicted ORFs: {len(final_orfs)}")
        print(f"Validated ORFs (True Positives): {len(validated_orfs)}")
        print(f"Unvalidated ORFs (False Positives): {len(unvalidated_orfs)}")
        if len(final_orfs) > 0:
            false_positive_rate = len(unvalidated_orfs) / len(final_orfs) * 100
            print(f"Estimated False Positive Rate: {false_positive_rate:.2f}%")
        else:
            print("No ORFs predicted, so no false positive rate to calculate.")


    except Exception as e:
        print(f"\nSkipping BLAST-related steps due to an error: {e}")
        print("Please ensure BLAST+ is installed and configured correctly in your Docker environment.")

    # Step 4: Estimate mean ORF length using geometric distribution
    calculate_expected_orf_length_geometric()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.4 MB/s eta 0:00:00
Extracted 50959 ORFs from 5497 sequences.
ORF length histogram saved to orf_lengths_histogram.png
Running BLASTp with query: extracted_orfs.fasta, database: /db/swissprot/swissprot
Error: 'blastp' command not found. Ensure BLAST+ is installed and in your system's PATH.
If using Docker, ensure your Dockerfile configures the PATH correctly and you are running within the container.

Skipping BLAST-related steps due to an error: [Errno 2] No such file or directory: 'blastp'
Please ensure BLAST+ is installed and configured correctly in your Docker environment.
Expected ORF length (codons, geometric model): 21.33
Expected ORF length (base pairs, geometric model): 64.00
